### implement sparse attention on your own in the GPT code that we wrote. Train on the data that you collected above:
Copy and paste the code here for the sparse attention that you wrote
share the training log (Epochs/x = 10 logs)
Share 10 examples of output

In [1]:
!pip install torchinfo
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 91.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 27.0 MB/s eta 0:00:00


In [2]:
!git clone "https://github.com/sunandhini96/School_of_AI-Assignment_11"

Cloning into 'School_of_AI-Assignment_11'...
remote: Enumerating objects: 54, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 54 (delta 9), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (54/54), 803.41 KiB | 1.77 MiB/s, done.


In [3]:
cd "/content/School_of_AI-Assignment_11/GPT/"

/content/School_of_AI-Assignment_11/GPT


In [4]:
from torchinfo import summary

In [5]:
from model import *
from utils import *

In [6]:
summary(model=SparseAttentionHead(head_size=8, num_embed=32, block_size=8, dropout=0.1,num_heads=4))

Layer (type:depth-idx)                   Param #
SparseAttentionHead                      --
├─Linear: 1-1                            1,024
├─Linear: 1-2                            1,024
├─Linear: 1-3                            1,024
├─Dropout: 1-4                           --
Total params: 3,072
Trainable params: 3,072
Non-trainable params: 0

In [7]:
import torch
from model import Transformer
from transformers import AutoTokenizer  # pip install transformers
from utils import (
    BATCH_SIZE,
    BLOCK_SIZE,
    DEVICE,
    DROPOUT,
    LEARNING_RATE,
    NUM_EMBED,
    NUM_HEAD,
    NUM_LAYER,
    MAX_ITER,
    EVAL_INTER,
    encode,
    decode,
    get_batch,
    save_model_to_chekpoint,
    estimate_loss,
)

# load model from checkpoint
# m = load_model_from_checkpoint(Transformer,vocab_size=vocab_size)

# example to decode sequence
# enc_sec = m.generate(idx=torch.zeros((1,1), dtype=torch.long),
# max_new_tokens=20)[0].tolist()
# print(decode(vocab=vocab, enc_sec=enc_sec))

# raw data
path_do_data = "/content/drive/MyDrive/sunandini/new_data.txt"
data_raw = open(path_do_data, encoding="utf-8").read()
# we use pretrained BERT tokenizer for performance improvements
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
vocab_size = tokenizer.vocab_size
# data_raw = data_raw[4000000:] # short dataset

# train/val split
data = encode(text_seq=data_raw, tokenizer=tokenizer)
n = int(0.9 * len(data))  # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# train a new model
model = Transformer(
    vocab_size=vocab_size,
    num_embed=NUM_EMBED,
    block_size=BLOCK_SIZE,
    num_heads=NUM_HEAD,
    num_layers=NUM_LAYER,
    dropout=DROPOUT,
)
# load model to GPU if available
m = model.to(DEVICE)
# print the number of parameters in the model
print(
    "Model with {:.2f}M parameters".format(sum(p.numel() for p in m.parameters()) / 1e6)
)


Token indices sequence length is longer than the specified maximum sequence length for this model (471768 > 512). Running this sequence through the model will result in indexing errors


Model with 89.48M parameters


In [9]:
# optimizer takes the model's parameters and the learning rate as input,
# and updates the parameters during the training process in order to
# minimize the loss function.

optimizer = torch.optim.AdamW(m.parameters(), lr=LEARNING_RATE)
MAX_ITER = 1000
EVAL_INTER=10
for step in range(0,MAX_ITER,10):

    # every EVAL_INTER evaluate the loss on train and val sets
    if step % EVAL_INTER == 0 or step == MAX_ITER - 1:
        loss_train = estimate_loss(
            data=train_data, model=m, block_size=BLOCK_SIZE, batch_size=BATCH_SIZE
        )
        loss_val = estimate_loss(
            data=val_data, model=m, block_size=BLOCK_SIZE, batch_size=BATCH_SIZE
        )
        print("step {:10} | train loss {:6.4f} | val loss {:6.4f}".format(step, loss_train, loss_val))

    # sample a batch of data
    xb, yb = get_batch(data=train_data, block_size=BLOCK_SIZE, batch_size=BATCH_SIZE)
    logits, loss = m.forward(xb, yb)
    # zero_grad() method sets the gradients of all parameters in the optimizer to zero
    optimizer.zero_grad(set_to_none=True)
    # backward() method on the loss variable calculates the gradients 
    # of the loss with respect to the model's parameters.
    loss.backward()
    # step() method on the optimizer updates the model's parameters 
    # using the calculated gradients, in order to minimize the loss.
    optimizer.step()



step          0 | train loss 3.7530 | val loss 7.0727
step         10 | train loss 3.9015 | val loss 7.3473
step         20 | train loss 3.8938 | val loss 7.0711
step         30 | train loss 3.9754 | val loss 7.1074
step         40 | train loss 3.9272 | val loss 7.0237
step         50 | train loss 3.9315 | val loss 6.8460
step         60 | train loss 3.8266 | val loss 6.9023
step         70 | train loss 3.9328 | val loss 6.8765
step         80 | train loss 3.9035 | val loss 6.8372
step         90 | train loss 3.8801 | val loss 6.8643
step        100 | train loss 3.8784 | val loss 6.8913
step        110 | train loss 3.8990 | val loss 7.0001
step        120 | train loss 3.9180 | val loss 6.9625
step        130 | train loss 3.8650 | val loss 6.9532
step        140 | train loss 3.8525 | val loss 6.9873
step        150 | train loss 3.9149 | val loss 6.9715
step        160 | train loss 3.9298 | val loss 6.8242
step        170 | train loss 3.8535 | val loss 6.9350
step        180 | train loss

In [10]:
save_model_to_chekpoint(model=m, path_to_checkpoint="checkpoint", epoch=step)

Successfully saved the model to checkpoint/checkpoint_epoch-990_11.04.2023_10:07:54.pt


In [11]:
# generate some output based on the context
context = torch.zeros((1, 1), dtype=torch.long, device=DEVICE)
print(
    decode(
        enc_sec=m.generate(idx=context, max_new_tokens=100, block_size=BLOCK_SIZE)[0],
        tokenizer=tokenizer,
    )
)

[PAD] or not bad the film, not probably. < br / > < br / > < br / > preston is never precisely by me, women and dragged proposed an actress have surprise two pilasters in dawson's oils and turns back to quietly empire inctuaries as his contract. " " " " and his ragged reagents, microbes = " " " " = = post @ - @ = = " " " " " = = december – west – aerodrome licensing = = = " "
